In [93]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder

In [94]:
df= pd.read_csv('../Data/Cleaned/mudita_laptops_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Brand      303 non-null    object 
 1   Series     303 non-null    object 
 2   Price      303 non-null    float64
 3   Ram        303 non-null    float64
 4   Storage    303 non-null    float64
 5   Graphics   303 non-null    object 
 6   Processor  303 non-null    object 
dtypes: float64(3), object(4)
memory usage: 16.7+ KB


In [ ]:
df.head(5)
df['Series'].value_counts()

Series
Aspire Series                   25
LOQ Series                      14
MacBook Air                     12
Legion Series                   12
No                              12
                                ..
GP66                             1
MacBook Pro Series (M3 Chip)     1
ROG Zephyrus Series              1
Alienware                        1
ZenBook Duo Series               1
Name: count, Length: 94, dtype: int64

In [96]:



categorical_feature = ['Brand','Series','Graphics','Processor']
numerical_feature = ['Ram','Storage']

scale=StandardScaler()
encode=OrdinalEncoder()

#Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scale, numerical_feature),
        ('cat', encode, categorical_feature)
    ])
df['Price']

0       32999.0
1       34999.0
2       34999.0
3       36999.0
4       45999.0
         ...   
298    535000.0
299    545000.0
300    569999.0
301    605000.0
302     88999.0
Name: Price, Length: 303, dtype: float64

In [97]:
X = df.drop('Price', axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', LinearRegression())])

df.isnull().sum()


Brand        0
Series       0
Price        0
Ram          0
Storage      0
Graphics     0
Processor    0
dtype: int64

In [98]:
#fit the pipeline to the training data
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Ram', 'Storage']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['Brand', 'Series',
                                                   'Graphics',
                                                   'Processor'])])),
                ('model', LinearRegression())])

In [99]:
# Find score
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

ValueError: Found unknown categories ['No'] in column 1 during transform